<a href="https://colab.research.google.com/github/AshKatzEm/SQLproject/blob/main/Notebook/Post_pipeline_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import numpy as np
import statistics as stat
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv')
df

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,...,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood
0,"Ed, Edd n Eddy: Scam of the Century",E,"Crude Humor,Mild Cartoon Violence",No Interactive Elements,No Rating Summary,Nintendo DS,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Napoleon Dynamite,E10+,"Cartoon Violence,Mild Language",No Interactive Elements,No Rating Summary,Nintendo DS,2007-10-31,2019-01-09T12:41:06,7 Studios,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Rail Simulator,E,No Descriptors,No Interactive Elements,No Rating Summary,Windows PC,2007-10-12,2019-01-09T12:41:06,Kuju Entertainment,Simulation,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NES Open Tournament Golf,E,No Descriptors,No Interactive Elements,No Rating Summary,Wii,1987-06-14,2019-07-12T12:10:28,Nintendo,Sports,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ultimate Duck Hunting,T,Mild Violence,No Interactive Elements,No Rating Summary,Wii,2009-05-26,2019-01-09T12:41:06,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45886,Turok 3: Shadow of Oblivion,M,"Blood and Gore,Violence",No Interactive Elements,This is a retro-style first-person shooter in ...,PlayStation 4,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45887,Cabela's® Big Game Hunter: Pro Hunts,T,"Blood,Violence",No Interactive Elements,This is a hunting simulation game in which pla...,Wii U,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45888,JoJo's Bizarre Adventure: All-Star Battle,T,"Animated Blood,Language,Mild Suggestive Themes...",Online Interactions Not Rated by the ESRB.,This is a fighting game in which players battl...,PlayStation 3,2013-08-29,2019-09-17T11:05:54,BANDAI NAMCO Entertainment America,Fighting,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45889,Titanfall,M,"Blood and Gore,Strong Language,Violence",Online Interactions Not Rated by the ESRB.,This is a team-based first-person shooter in w...,Xbox One,2014-03-11,2020-11-30T09:29:04,"Electronic Arts, Respawn Entertainment, Bluepo...","Action, Shooter",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# We need to create a program which takes the values in the descriptors column and creates columns out of them
df['Number of Descriptors'] = 0;
for r in range(len(df)):
  # What descriptors does the game have?
  descriptors = df.loc[r, "content_descriptors"].split(",")
  # How many descriptors does it have?
  df.loc[r, "Number of Descriptors"] = len(descriptors)

  for i in descriptors:
    if i == 'No Content Descriptors' or i == 'No Descriptors':
      # No need for a no descriptors column
      df.loc[r,'Number of Descriptors'] = 0

    else:
      if i in df.columns:
        # Hot-encode 1 for each of the descriptors this game has.
        df.loc[r, i] = 1

      else:
        # Hot-encode 1 for each of the descriptors this game has.
        df.loc[r, i] = 1
        df[i] = df[i].fillna(0)

df.columns

Index(['game_title', 'esrb_rating', 'content_descriptors',
       'interactive_elements', 'content_summary', 'platform', 'release_date',
       'updated', 'developers', 'genres', 'release_year', 'publisher',
       'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'global_sales',
       'Number of Descriptors', 'Crude Humor', 'Mild Cartoon Violence',
       'Cartoon Violence', 'Mild Language', 'Mild Violence', 'Violence',
       'Blood and Gore', 'Language', 'Nudity', 'Sexual Themes',
       'Mild Fantasy Violence', 'Alcohol Reference', 'Mild Suggestive Themes',
       'Simulated Gambling', 'Fantasy Violence', 'Mild Blood',
       'Use of Alcohol', 'Use of Tobacco', 'Drug Reference', 'Strong Language',
       'Suggestive Themes', 'Blood', 'Partial Nudity', 'Tobacco Reference',
       'Comic Mischief', 'Animated Blood', 'Intense Violence',
       'Alcohol and Tobacco Reference', 'Violent References',
       'Strong Sexual Content', 'Use of Alcohol and Tobacco', 'Use of Drugs',
       '

In [ ]:
descriptorCols = df.columns[17:len(df.columns)]
for i in descriptorCols:
  print(df[i].value_counts())


Number of Descriptors
1    15638
0    12831
2    11139
3     8855
4     5763
5     3996
6     1383
7      223
8        2
Name: count, dtype: int64
Crude Humor
0.0    57406
1.0     2424
Name: count, dtype: int64
Mild Cartoon Violence
0.0    57787
1.0     2043
Name: count, dtype: int64
Cartoon Violence
0.0    57097
1.0     2733
Name: count, dtype: int64
Mild Language
0.0    53450
1.0     6380
Name: count, dtype: int64
Mild Violence
0.0    56110
1.0     3720
Name: count, dtype: int64
Violence
0.0    44854
1.0    14976
Name: count, dtype: int64
Blood and Gore
0.0    53542
1.0     6288
Name: count, dtype: int64
Language
0.0    54729
1.0     5101
Name: count, dtype: int64
Nudity
0.0    59185
1.0      645
Name: count, dtype: int64
Sexual Themes
0.0    56363
1.0     3467
Name: count, dtype: int64
Mild Fantasy Violence
0.0    57112
1.0     2718
Name: count, dtype: int64
Alcohol Reference
0.0    56915
1.0     2915
Name: count, dtype: int64
Mild Suggestive Themes
0.0    56143
1.0     3687
Name: c

In [ ]:
# df.to_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv', index=False)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv')
df

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,...,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood
0,"Ed, Edd n Eddy: Scam of the Century",E,"Crude Humor,Mild Cartoon Violence",No Interactive Elements,No Rating Summary,Nintendo DS,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Napoleon Dynamite,E10+,"Cartoon Violence,Mild Language",No Interactive Elements,No Rating Summary,Nintendo DS,2007-10-31,2019-01-09T12:41:06,7 Studios,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Rail Simulator,E,No Descriptors,No Interactive Elements,No Rating Summary,Windows PC,2007-10-12,2019-01-09T12:41:06,Kuju Entertainment,Simulation,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NES Open Tournament Golf,E,No Descriptors,No Interactive Elements,No Rating Summary,Wii,1987-06-14,2019-07-12T12:10:28,Nintendo,Sports,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ultimate Duck Hunting,T,Mild Violence,No Interactive Elements,No Rating Summary,Wii,2009-05-26,2019-01-09T12:41:06,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45886,Turok 3: Shadow of Oblivion,M,"Blood and Gore,Violence",No Interactive Elements,This is a retro-style first-person shooter in ...,PlayStation 4,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45887,Cabela's® Big Game Hunter: Pro Hunts,T,"Blood,Violence",No Interactive Elements,This is a hunting simulation game in which pla...,Wii U,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45888,JoJo's Bizarre Adventure: All-Star Battle,T,"Animated Blood,Language,Mild Suggestive Themes...",Online Interactions Not Rated by the ESRB.,This is a fighting game in which players battl...,PlayStation 3,2013-08-29,2019-09-17T11:05:54,BANDAI NAMCO Entertainment America,Fighting,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45889,Titanfall,M,"Blood and Gore,Strong Language,Violence",Online Interactions Not Rated by the ESRB.,This is a team-based first-person shooter in w...,Xbox One,2014-03-11,2020-11-30T09:29:04,"Electronic Arts, Respawn Entertainment, Bluepo...","Action, Shooter",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


this is getting rid of the duplicates which are clearly present

In [ ]:
df = df.drop_duplicates(subset=['game_title', 'esrb_rating' , 'content_descriptors' ,	'interactive_elements','content_summary' , 'platform'], keep='first')
df.reset_index(drop=True, inplace=True)
len(df)

52274

In [ ]:
# get rid of games where platform is Nan
df = df.drop(df[df['platform'].isna()].index)
df.reset_index(drop=True, inplace=True)
len(df)

52271

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv')
df.iloc[0:10]

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,...,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood
0,"Ed, Edd n Eddy: Scam of the Century",E,"Crude Humor,Mild Cartoon Violence",No Interactive Elements,No Rating Summary,Nintendo DS,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Napoleon Dynamite,E10+,"Cartoon Violence,Mild Language",No Interactive Elements,No Rating Summary,Nintendo DS,2007-10-31,2019-01-09T12:41:06,7 Studios,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Rail Simulator,E,No Descriptors,No Interactive Elements,No Rating Summary,Windows PC,2007-10-12,2019-01-09T12:41:06,Kuju Entertainment,Simulation,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NES Open Tournament Golf,E,No Descriptors,No Interactive Elements,No Rating Summary,Wii,1987-06-14,2019-07-12T12:10:28,Nintendo,Sports,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ultimate Duck Hunting,T,Mild Violence,No Interactive Elements,No Rating Summary,Wii,2009-05-26,2019-01-09T12:41:06,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Fortnite - Anime Legends,T,Violence,"Users Interact, In-Game Purchases.",This is an action game in which players build ...,PlayStation 4,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Demoniaca: Everlasting Night,M,"Blood and Gore,Language,Nudity,Violence",No Interactive Elements,No Rating Summary,Nintendo Switch,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Ryan’s Rescue Squad,E,Mild Cartoon Violence,No Interactive Elements,This is a side-scrolling platformer in which p...,Windows PC,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,The Sims™ 4 - Wedding Stories,T,"Crude Humor,Sexual Themes,Violence",Online Interactions Not Rated by the ESRB.,No Rating Summary,Windows PC,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,The400 Mini,E,Mild Fantasy Violence,No Interactive Elements,This is a collection of classic Atari 400 game...,The400 Mini,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cg = pd.read_csv('/content/drive/MyDrive/SQLproject/SupplementalData/computer_games.csv')
cg.iloc[0:10]

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
1,A-Train,Artdink,"Artdink, Maxis, Ocean Software",Vehicle Simulation Game,"Windows, Mac",1985
2,A-10 Cuba!,Parsoft Interactive,Activision,Flight simulator,"Windows, Mac","November 30, 1996"
3,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996"
4,A.D.A.M. Life's Greatest Mysteries,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1996
5,A.D.A.M. The Inside Story,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1995
6,A.R.E.S.: Extinction Agenda,Extend Studio,"Origo Games, Aksys Games","Action, platform",Microsoft Windows,"December 14, 2010"
7,A Girls Fabric Face,Stanislaw Truchowskit,TurnVex,Adventure,Microsoft Windows,"March 10, 2017"
8,A Hat in Time,Gears for Breakfast,Gears for Breakfast,"Platform, action-adventure","macOS, Microsoft Windows","October 5, 2017"
9,A Game of Thrones: Genesis,Cyanide,Focus Home Interactive,Strategy,Microsoft Windows,"September 28, 2011"


In [ ]:
gam = cg['Name']
esrb = df['game_title']
match = {}
for i in gam:
  if i in esrb.values:
    match[i] = [esrb[esrb == i].index[0], gam[gam == i].index[0]]


In [ ]:
df['release_date'].notna().sum()

17807

In [ ]:
# these are the places where a game is on PC
pcI = df[df['platform'] == 'Windows PC'].index

#these are the games which match with the pc dataset
for n in match.keys():
  # these are the places where a particular game appears in esrb
  nameI = df[df['game_title']==n ].index
  # this is where that game appears ON PC in esrb
  inter = nameI.intersection(pcI)
  if len(inter) > 0:
    # for each pc game index
    for i in inter:
      if df.loc[i,'release_date'] == 'nan':
        df.loc[i, 'release_date'] = cg.loc[match[n][1], 'Date Released']

      # get the Year out of the CG Date Released cell
      date = str(cg.loc[match[n][1], 'Date Released'])
      year = int(re.search(r"\d\d\d\d", date).group())
      # set the release year of that pc game index to year
      df.loc[i,'release_year'] = year



In [ ]:
# get the years from release date
for i in range(len(df)):
  df.loc[i,'release_date_year'] = str(df.loc[i,'release_date'])[0:4]

df['release_date_year'].value_counts()

,count
release_date_year,
nan,28084
2009,1270
2016,1171
2010,1159
2015,1144
2017,1081
2014,1073
2018,995
2008,990


In [ ]:
df['release_year'].notna().sum()

7396

In [ ]:
# try to fill any Nans in release years with those years
df['release_year'] = df['release_year'].fillna(df['release_date_year'])

In [ ]:
len(df[df['release_year']!='nan'])

22513

In [ ]:
pd.set_option('display.max_rows', 100)
df.platform.unique()

array(['Nintendo DS', 'Windows PC', 'Wii', 'PlayStation 4',
       'Nintendo Switch', 'The400 Mini', 'Xbox One', 'PS Vita',
       'Nintendo 3DS', 'PSP', 'Xbox 360', 'PlayStation 3', 'Xbox',
       'Wii U', 'macOS', 'GameCube', 'PlayStation 5', 'PlayStation 2',
       'Nintendo DSi', 'Xbox Series', 'Online', 'Atari 2600', 'Linux',
       'Game Boy Advance', 'Web Browser', 'Stadia', 'PlayStation 1',
       'Steam', 'Atari 7800', 'DVD', 'Palm Pilot', 'Pocket PC',
       'Nintendo 64', 'Game Boy Color', 'Sega Dreamcast', 'SEGA Saturn',
       'Intellivision Amico', 'Plug-and-Play', 'Hyperscan',
       'Mobile Phone', 'N-Gage', 'Game Boy', 'Sega Genesis', 'SNES',
       'Cable Box', 'Game Gear', 'NES', 'Neo Geo Pocket Color',
       'Windows Phone', 'PlayStation Certified Device', 'IPTV', 'iOS',
       'Palm', 'Win CE', 'Windows CE', 'SEGA CD', '3DO',
       'ModRetro Chromatic', 'Game & Watch', 'Zodiac', 'Android',
       'Pokemon Mini', 'Gizmondo', 'Digital TV', 'Virtual Boy', 'NUON',
  

In [ ]:
# Does the title have a year in it
# Drop nan platforms
# get console ranges
console_range = {
  '3DO': [1993, 1996, 5],
 'Amazon': [2011, 2024, 9],
 'Android': [2008, 2024, 9],
 'Atari 2600': [1977, 1992, 2],
 'Atari 7800': [1986, 1992, 3],
 'Atari Jaguar': [1993, 1996, 5],
 'Cable Box': [2000, 2020, 7],
 "Connected TV's": [2008, 2024, 8],
 'DVD': [1995, 2023, 6],
 'Digital TV': [2007, 2020, 7],
 'Game & Watch': [1980, 1991, 2],
 'Game Boy': [1989, 2003, 4],
 'Game Boy Advance': [2001, 2008, 6],
 'Game Boy Color': [1998, 2003, 5],
 'Game Gear': [1990, 2000, 4],
 'GameCube': [2001, 2007, 6],
 'Genesis': [1989, 1997, 4],
 'Gizmondo': [2005, 2006, 7],
 'Hyperscan': [2006, 2007, 7],
 'IPTV': [2001, 2024, 8],
 'Intellivision Amico': [2021, 2024, 9],
 'Linux': [1991, 2025, 9],
 'Linux based PC': [1991, 2025, 9],
 'Meta Quest': [2019, 2024, 8],
 'Meta Quest 2': [2020, 2025, 9],
 'Microsoft Teams': [2017, 2024, 9],
 'Mini Console': [2017, 2024, 9],
 'Mobile (GooglePlay)': [2012, 2025, 9],
 'Mobile Phone': [1996, 2024, 9],
 'ModRetro Chromatic': [2024, 2025, 9],
 'N-Gage': [2003, 2006, 6],
 'NES': [1983, 1995, 3],
 'NUON': [2000, 2004, 6],
 'Neo Geo Pocket Color': [1999, 2001, 5],
 'New Nintendo 3DS': [2012, 2020, 8],
 'Nintendo 3DS': [2011, 2020, 8],
 'Nintendo 64': [1996, 2002, 5],
 'Nintendo DS': [2004, 2014, 7],
 'Nintendo DSi': [2008, 2014, 7],
 'Nintendo Switch': [2017, 2025, 9],
 'Online': [1996, 2025, 9],
 'PS Vita': [2012, 2019, 8],
 'PSP': [2005, 2014, 7],
 'Palm': [1996, 2010, 5],
 'Palm Pilot': [1996, 2010, 5],
 'PlayStation 1': [1995, 2006, 5],
 'PlayStation 2': [2000, 2014, 6],
 'PlayStation 3': [2006, 2018, 7],
 'PlayStation 4': [2013, 2024, 8],
 'PlayStation 5': [2020, 2025, 9],
 'PlayStation Certified Device': [2011, 2024, 9],
 'Plug-and-Play': [1994, 2020, 6],
 'Pocket PC': [2000, 2008, 5],
 'Pokemon Mini': [2001, 2002, 4],
 'SEGA CD': [1992, 1996, 4],
 'SEGA Saturn': [1994, 1999, 5],
 'SNES': [1990, 2000, 4],
 'Samsung TV': [2008, 2024, 8],
 'Samsung TV (model Pavv Bordo)': [2008, 2024, 8],
 'Samsung TV 750': [2008, 2024, 8],
 'Sega 32X': [1994, 1996, 5],
 'Sega Dreamcast': [1998, 2004, 6],
 'Sega Genesis': [1989, 1999, 4],
 'Sifteo Cubes': [2011, 2014, 7],
 'Stadia': [2019, 2025, 9],
 'Steam': [2013, 2025, 9],
 'The400 Mini': [2024, 2025, 9],
 'VG Pocket': [2006, 2008, 6],
 'Virtual Boy': [1995, 1996, 5],
 'Web Browser': [1996, 2024, 9],
 'Web TV': [1996, 2013, 6],
 'Wii': [2006, 2014, 7],
 'Wii U': [2012, 2018, 8],
 'Win CE': [1996, 2013, 6],
 'Windows CE': [1996, 2013, 6],
 'Windows PC': [1994, 2024, 9],
 'Windows Phone': [2010, 2017, 7],
 'Xbox': [2001, 2008, 6],
 'Xbox 360': [2005, 2016, 7],
 'Xbox One': [2013, 2024, 8],
 'Xbox Series': [2020, 2025, 9],
 'Zodiac': [2003, 2005, 6],
 'iOS': [2007, 2024, 9],
 'macOS': [2001, 2025, 9]
                 }

In [ ]:
#console_range[][0].sort_values()
dict(sorted(console_range.items(), key=lambda item: item[1]))

{'Atari 2600': [1977, 1992],
 'Game & Watch': [1980, 1991],
 'NES': [1983, 1995],
 'Atari 7800': [1986, 1992],
 'Genesis': [1989, 1997],
 'Sega Genesis': [1989, 1999],
 'Game Boy': [1989, 2003],
 'SNES': [1990, 2000],
 'Game Gear': [1990, 2000],
 'Linux': [1991, 2025],
 'Linux based PC': [1991, 2025],
 'SEGA CD': [1992, 1996],
 '3DO': [1993, 1996],
 'Atari Jaguar': [1993, 1996],
 'Sega 32X': [1994, 1996],
 'SEGA Saturn': [1994, 1999],
 'Windows PC': [1994, 2024],
 'Virtual Boy': [1995, 1996],
 'PlayStation 1': [1995, 2006],
 'Plug-and-Play': [1995, 2006],
 'DVD': [1995, 2023],
 'Nintendo 64': [1996, 2002],
 'Palm Pilot': [1996, 2010],
 'Palm': [1996, 2010],
 'Win CE': [1996, 2013],
 'Windows CE': [1996, 2013],
 'Web TV': [1996, 2013],
 'Web Browser': [1996, 2024],
 'Mobile Phone': [1996, 2024],
 'Online': [1996, 2025],
 'Game Boy Color': [1998, 2003],
 'Sega Dreamcast': [1998, 2004],
 'Neo Geo Pocket Color': [1999, 2001],
 'NUON': [2000, 2004],
 'Pocket PC': [2000, 2008],
 'PlayStation

In [ ]:
df['generation'] = 0
for i in range(len(df)):
  df.loc[i,'generation'] = console_range[df.loc[i,'platform']][2]

In [ ]:
pd.set_option("display.max_columns", None)
df[df['release_year']=='nan']

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year
0,"Ed, Edd n Eddy: Scam of the Century",E,"Crude Humor,Mild Cartoon Violence",No Interactive Elements,No Rating Summary,Nintendo DS,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
5,Fortnite - Anime Legends,T,Violence,"Users Interact, In-Game Purchases.",This is an action game in which players build ...,PlayStation 4,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
6,Demoniaca: Everlasting Night,M,"Blood and Gore,Language,Nudity,Violence",No Interactive Elements,No Rating Summary,Nintendo Switch,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
7,Ryan’s Rescue Squad,E,Mild Cartoon Violence,No Interactive Elements,This is a side-scrolling platformer in which p...,Windows PC,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
8,The Sims™ 4 - Wedding Stories,T,"Crude Humor,Sexual Themes,Violence",Online Interactions Not Rated by the ESRB.,No Rating Summary,Windows PC,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52260,Contra: Operation Galuga,T,"Fantasy Violence,Mild Blood,Mild Language",No Interactive Elements,This is an action platformer in which players ...,Xbox One,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
df['platform']= df['platform'].astype(str)

In [ ]:
dup = df[df.duplicated(subset=['game_title','platform'], keep= 'first')]
dup[dup['game_title']=='Need for Speed™ Rivals']
# first = dup[dup['game_title']=='Need for Speed™ Rivals']['platform'].iloc[0]
# second = dup[dup['game_title']=='Need for Speed™ Rivals']['platform'].iloc[2]
# if first == second:
#   print(first,'=',second)
# else:
#   print(first,'!=',second)
#27, 1503

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year
20240,Need for Speed™ Rivals,E10+,Mild Violence,Online Interactions Not Rated by the ESRB.,This is an arcade-style racing game in which p...,Xbox 360,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
26835,Need for Speed™ Rivals,E10+,Mild Violence,Online Interactions Not Rated by the ESRB.,This is an arcade-style racing game in which p...,PlayStation 4,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
28824,Need for Speed™ Rivals,E10+,Mild Violence,Online Interactions Not Rated by the ESRB.,This is an arcade-style racing game in which p...,Xbox One,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
34289,Need for Speed™ Rivals,E10+,Mild Violence,Online Interactions Not Rated by the ESRB.,This is an arcade-style racing game in which p...,Windows PC,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
52269,Need for Speed™ Rivals,E10+,Mild Violence,Online Interactions Not Rated by the ESRB.,This is an arcade-style racing game in which p...,PlayStation 3,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan


In [ ]:
df = df.drop_duplicates(subset=['game_title','platform','esrb_rating' , 'content_descriptors' ,'interactive_elements' ], keep='first')
df.reset_index(drop=True, inplace=True)
len(df)

45891

it doesn't seem to be matching platform to platform.
or you don't understand what duplicate is doing.
<br>
Why are there five rows above? they are not duplicates

In [ ]:
len(df[df['release_year']!='nan'])

23443

In [ ]:
pd.set_option('display.max_rows', 100)

df.sort_values('game_title', ascending=False).iloc[[6484, 38168, 43829]]

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year
20934,The Legend of Korra™: A New Era Begins,E10+,Cartoon Violence,No Interactive Elements,"This is a strategy game, based on the TV show ...",Nintendo 3DS,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
43934,Cities XL 2012,E,Mild Cartoon Violence,No Interactive Elements,This is a simulation game in which players can...,Windows PC,2011-10-20,2020-06-11T12:02:05,Focus Home Interactive,Strategy,2011.0,Focus Home Interactive,0.01,0.05,0.0,0.01,0.07,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2011
32002,Amnesia: Memories,T,"Blood,Language,Sexual Themes,Use of Alcohol,Vi...",No Interactive Elements,This is a visual novel in which players follow...,Nintendo Switch,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan


In [ ]:
# if the year is in the title
for i in range(len(df)):
  if df.loc[i,'release_year'] == 'nan':
    name = str(df.loc[i, 'game_title'])
    #titlenum =
    if re.search(r"\d\d\d\d", name):
      titlenum = int(re.search(r"\d\d\d\d", name).group())
      if titlenum >=1970 and titlenum <= 2025:
        df.loc[i,'release_year'] = titlenum
        print(name, titlenum)

In [ ]:
len(df[df['release_year']!='nan'])

20448

In [ ]:
# if two digits of the year are in the title, it's  a sports game, find 21st century sports games
sprts = ["wwe", 'sports', 'nba', 'fifa', 'ncaa', 'nfl', 'soccer', 'baseball', 'basketball', 'pga', 'nhl', 'nascar', 'hockey', 'football' ]
for i in range(len(df)):
  if df.loc[i,'release_year'] == 'nan':
    name = str(df.loc[i, 'game_title'])
    #titlenum =
    if re.search(r"\d\d", name):
      titlenum = re.search(r"\d\d", name).group()
      if int(titlenum) >=1 and int(titlenum) <= 25:
        for n in sprts:
          if n in name.lower():
            df.loc[i,'release_year'] = int("20" + titlenum)
            print(name, titlenum)
            break

In [ ]:
# if two digits of the year are in the title, it's  a sports game, find 20th century sports games
sprts = ["wwe", 'sports', 'nba', 'fifa', 'ncaa', 'nfl', 'soccer', 'baseball', 'basketball', 'pga', 'nhl', 'nascar', 'hockey', 'football' ]
for i in range(len(df)):
  if df.loc[i,'release_year'] == 'nan':
    name = str(df.loc[i, 'game_title'])
    #titlenum =
    if re.search(r"\d\d", name):
      titlenum = re.search(r"\d\d", name).group()
      if int(titlenum) >=70 and int(titlenum) <= 99:
        for n in sprts:
          if n in name.lower():
            df.loc[i,'release_year'] = int("19" + titlenum)
            print(name, titlenum)
            break

In [ ]:
len(df[df['release_year']!='nan'])

20851

In [ ]:
# Here, we are trying to fill some release year rows from a game on one console which we have data on-
# -to the other consoles that game is on which we don't have a release date for
#find the games where on one console it has a release year but others it doesn't
# for each of those other games, check that the year fits within it's console years, assign it if yes
# if not, take the median year of the consoles life span
countrows = 0
countFullrows = 0
for i in range(len(df)):
  countrows+=1

  if countrows%1000 == 0:
    print(countrows, countFullrows)
  try:
    # if this is a row which has already been filled
    if df.loc[i,'release_year'] != 'nan':

      # Get the already filled in release year
      fullYear = int(df.loc[i,'release_year'])
      countFullrows+=1

      # Are there any versions of that game which arent filled in?
      # get the details to check for other versions
      name = df.loc[i,'game_title']
      rating = df.loc[i,'esrb_rating']
      intelements = df.loc[i,'interactive_elements']
      summary = df.loc[i,'content_summary']
      # Check for other versions
      nameOccurences = df[df['game_title']==name].index
      ratingOccurences = df[ df['esrb_rating']==rating ].index
      intelementsOccurences = df[ df['interactive_elements']==intelements ].index
      summaryOccurences = df[ df['content_summary']==summary ].index
      releaseYearOccurences = df[ df['release_year']=='nan' ].index
      # print(name, rating, intelements, summary)
      # print("name: ", nameOccurences)
      # print("rating: ", ratingOccurences)
      # print("intelements: ", intelementsOccurences)
      # print("summary: ", summaryOccurences)
      # print("releaseYear: ", releaseYearOccurences)
      intersect1 = nameOccurences.intersection(ratingOccurences)
      intersect2 = intersect1.intersection(intelementsOccurences)
      intersect3 = intersect2.intersection(summaryOccurences)
      # print(intersect1)
      # print(intersect2)
      # print(intersect3)
      overlap = intersect3.intersection(releaseYearOccurences)


      if len(overlap) > 0:
        # get the rage of a console in the nans
        for nan in overlap:
          rel = console_range[df.loc[nan,'platform']][0]
          dis = console_range[df.loc[nan,'platform']][1]
          years = range(rel,dis)

          # If fullYear is after than the console release date and before than the discontiue date
          if fullYear >= rel and fullYear <= dis:
            # it will work for this game
            df.loc[nan,'release_year'] = fullYear

          # If we have a release date on one console which is before the console we're looking at came out, or
          # If we have a release date on one console which is after the console we're looking at was discontinued
          elif fullYear < rel or fullYear > dis:
            # We have data on an Older game, and we're trying to fill in the data for the version which was ported to a new console, or
            # We have data on a New game and we're trying to fill in the data for the old version.
            df.loc[nan,'release_year'] = round(stat.median(years))


  except Exception as e:
    print(e)
    continue



len(df[df['release_year']!='nan'])


1000 470
2000 907
3000 1358
4000 1822
5000 2325
6000 2754
7000 3219
8000 3674
9000 4117
10000 4592
11000 5080
12000 5547
13000 6017
14000 6466
15000 6920
16000 7384
17000 7865
18000 8342
19000 8845
20000 9304
21000 9818
22000 10316
23000 10821
24000 11315
25000 11782
26000 12299
27000 12813
28000 13313
29000 13837
30000 14350
31000 14867
32000 15378
33000 15909
34000 16365
35000 16868
36000 17351
37000 17842
38000 18363
39000 18877
40000 19389
41000 19900
42000 20394
43000 20919
44000 21421
45000 21921


23443

1 overlap:  Index([10813], dtype='int64') <br>
4 overlap:  Index([24177], dtype='int64') <br>
10 overlap:  Index([22693], dtype='int64') <br>
61 overlap:  Index([10698, 10699, 10757, 28954, 37255], dtype='int64') <br>
94 overlap:  Index([6919], dtype='int64') <br>
115 overlap:  Index([45831], dtype='int64') <br>
129 overlap:  Index([15452, 40592], dtype='int64') <br>
184 overlap:  Index([26751], dtype='int64') <br>
198 overlap:  Index([1139, 1546, 16530, 41713, 43287], dtype='int64') <br>
199 overlap:  Index([32058], dtype='int64') <br>
220 overlap:  Index([888, 26776], dtype='int64') <br>
221 overlap:  Index([5171], dtype='int64') <br>
224 overlap:  Index([9898, 40706, 43315], dtype='int64') <br>
225 overlap:  Index([226, 15542], dtype='int64') <br>
230 overlap:  Index([44162], dtype='int64') <br>
237 overlap:  Index([26672], dtype='int64') <br>
241 overlap:  Index([44158], dtype='int64') <br>
248 overlap:  Index([32106], dtype='int64') <br>
249 overlap:  Index([32419, 43611], dtype='int64') <br>
284 overlap:  Index([11263, 16901, 39173], dtype='int64') <br>

In [ ]:
df.iloc[[284, 11263, 16901, 39173]]

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year,generation
284,Galaga,E,No Descriptors,No Interactive Elements,No Rating Summary,Xbox 360,2006-07-26,2020-12-21T02:42:00,Bandai Namco Entertainment,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2006,7
11263,Galaga,E,No Descriptors,No Interactive Elements,No Rating Summary,Game Boy Color,NaN,NaN,NaN,NaN,2000,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,5
16901,Galaga,E,Mild Animated Violence,No Interactive Elements,No Rating Summary,PlayStation 1,NaN,NaN,NaN,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,5
39173,Galaga,E,Mild Animated Violence,No Interactive Elements,No Rating Summary,Windows PC,NaN,NaN,NaN,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,9


In [ ]:
ind= df[df['generation'] == 4].index
ind.intersection([1976, 2215, 3210])

Index([1976, 2215], dtype='int64')

In [ ]:
# df.to_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv', index=False)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv')
# try to fill any Nans in release years with those years
df['release_year'] = df['release_year'].fillna('nan')
df

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year,generation
0,"Ed, Edd n Eddy: Scam of the Century",E,"Crude Humor,Mild Cartoon Violence",No Interactive Elements,No Rating Summary,Nintendo DS,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,7
1,Napoleon Dynamite,E10+,"Cartoon Violence,Mild Language",No Interactive Elements,No Rating Summary,Nintendo DS,2007-10-31,2019-01-09T12:41:06,7 Studios,NaN,2007.0,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2007.0,7
2,Rail Simulator,E,No Descriptors,No Interactive Elements,No Rating Summary,Windows PC,2007-10-12,2019-01-09T12:41:06,Kuju Entertainment,Simulation,2007.0,Electronic Arts,0.00,0.01,0.00,0.00,0.01,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2007.0,9
3,NES Open Tournament Golf,E,No Descriptors,No Interactive Elements,No Rating Summary,Wii,1987-06-14,2019-07-12T12:10:28,Nintendo,Sports,1987.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1987.0,7
4,Ultimate Duck Hunting,T,Mild Violence,No Interactive Elements,No Rating Summary,Wii,2009-05-26,2019-01-09T12:41:06,NaN,NaN,2009.0,Zoo Digital Publishing,0.07,0.00,0.00,0.01,0.08,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45886,Turok 3: Shadow of Oblivion,M,"Blood and Gore,Violence",No Interactive Elements,This is a retro-style first-person shooter in ...,PlayStation 4,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
# Here were trying to fill the release year columns where no versions of that game have release year data
countrows = 0
countNanrows = 0
for i in range(len(df)):
  countrows+=1
  if countrows%1000 == 0:
    print(countrows, countNanrows)
  try:
    # if this is a row we need to fill. At this point, if one of this game is nan, all of this game should be nan
    if df.loc[i,'release_year'] == 'nan':

      countNanrows+=1

      # get the details of this game to check for other versions
      name = df.loc[i,'game_title']
      rating = df.loc[i,'esrb_rating']
      intelements = df.loc[i,'interactive_elements']
      summary = df.loc[i,'content_summary']
      # Check if this game is on other consoles
      nameOccurences = df[df['game_title']==name].index
      ratingOccurences = df[ df['esrb_rating']==rating ].index
      intelementsOccurences = df[ df['interactive_elements']==intelements ].index
      summaryOccurences = df[ df['content_summary']==summary ].index
      releaseYearOccurences = df[ df['release_year']=='nan' ].index
      # Find the intersection of these conditions
      intersect1 = nameOccurences.intersection(ratingOccurences)
      intersect2 = intersect1.intersection(intelementsOccurences)
      intersect3 = intersect2.intersection(summaryOccurences)
      overlap = intersect3.intersection(releaseYearOccurences)


      # what consoles is it on?
      consoles = df.loc[overlap,'platform'].values

      # create a list of all of the console-release_years of the consoles which that game is on, CRY
      CRY = []
      # Create a list of all of the console-discontinued_years, CDY
      CDY = []
      for c in overlap:
        console = df.loc[c, 'platform']
        CRY.append(console_range[console][0])
        CDY.append(console_range[console][1])

      # Find the max of CRY
      # Find the min of CDY
      bot = max(CRY)
      top = min(CDY)
      years = range(bot,top)

      # If this game had one simultaneous console release
      if bot < top and len(years)>0:
        # All of the consoles can have the same release date
        year = round(stat.median(years))
        # get the median of those years and set it as the release year for all entries of that game
        df.loc[overlap,'release_year'] = year

      # If this game (probably) came out the year one console was ending and one console was begginging
      elif (bot == top and bot>0):
        print(overlap, bot, top, " Barely in range" )
        for gen in range(1,10):
          try:
            agen = df[df['generation'] == gen].index
            group = agen.intersection(overlap)
            CRY = []
            CDY = []
            for c in group:
              console = df.loc[c, 'platform']
              CRY.append(console_range[console][0])
              CDY.append(console_range[console][1])

            bot = max(CRY)
            top = min(CDY)
            years = range(bot,top)
            year = round(stat.median(years))
            print('gen',gen, 'group', group, bot, top, year)
            df.loc[group,'release_year'] = year

          except Exception as e:
            continue


      # If one of the consoles which a game is on came out after a different console the game is one was discontinued
      # This means that game was release at least twice, with a decent gap in between. Probably all remakes.
      elif bot > top:
        # All of the consoles can NOT have the same release date. Find each console's median date and set it to that because its a garaunteed valid date which is can't be far from the true date
        print(overlap, bot, top, " out of range")
        for gen in range(1,10):
          try:
            agen = df[df['generation'] == gen].index
            group = agen.intersection(overlap)
            CRY = []
            CDY = []
            for c in group:
              console = df.loc[c, 'platform']
              CRY.append(console_range[console][0])
              CDY.append(console_range[console][1])

            bot = max(CRY)
            top = min(CDY)
            years = range(bot,top)
            year = round(stat.median(years))
            print('gen',gen, 'group', group, bot, top, year)
            df.loc[group,'release_year'] = year

          except Exception as e:
            continue

      else:
        print('FAILED', name, consoles,"bot", bot, 'top', top ,'years',years)
        continue
      # set the game_release_date for all those games to median year of CRY-CDY

  except Exception as e:
    print(e)
    continue



len(df[df['release_year']!='nan'])


Index([517, 32417], dtype='int64') 2011 2003  out of range
gen 5 group Index([517], dtype='int64') 1998 2003 2000
gen 8 group Index([32417], dtype='int64') 2011 2020 2015
Index([586, 20460], dtype='int64') 2011 2003  out of range
gen 4 group Index([20460], dtype='int64') 1989 2003 1996
gen 8 group Index([586], dtype='int64') 2011 2020 2015
Index([587, 15956, 36700, 36701, 37801, 42895], dtype='int64') 2012 1999  out of range
gen 5 group Index([587, 37801], dtype='int64') 1995 1999 1996
gen 7 group Index([36700, 42895], dtype='int64') 2006 2014 2010
gen 8 group Index([36701], dtype='int64') 2012 2019 2015
gen 9 group Index([15956], dtype='int64') 1994 2024 2008
Index([745, 36424, 38195, 39666], dtype='int64') 2012 2006  out of range
gen 5 group Index([36424], dtype='int64') 1995 2006 2000
gen 7 group Index([745, 39666], dtype='int64') 2006 2014 2010
gen 8 group Index([38195], dtype='int64') 2012 2019 2015
1000 452
Index([1873, 13109, 14960, 15228, 21981, 29491, 41993], dtype='int64') 20

45891

In [ ]:
df.iloc[[587, 15956, 36700, 36701, 37801, 42895]]

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year,generation
587,MEGA MAN X4,E,"Animated Blood,Animated Violence",No Interactive Elements,No Rating Summary,SEGA Saturn,NaN,NaN,NaN,NaN,1996,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5
15956,MEGA MAN X4,E,"Animated Blood,Animated Violence",No Interactive Elements,No Rating Summary,Windows PC,NaN,NaN,NaN,NaN,2008,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,9
36700,MEGA MAN X4,E,"Animated Blood,Violence",No Interactive Elements,No Rating Summary,PlayStation 3,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,7
36701,MEGA MAN X4,E,"Animated Blood,Violence",No Interactive Elements,No Rating Summary,PS Vita,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8
37801,MEGA MAN X4,E,"Animated Blood,Animated Violence",No Interactive Elements,No Rating Summary,PlayStation 1,NaN,NaN,NaN,NaN,1996,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5
42895,MEGA MAN X4,E,"Animated Blood,Violence",No Interactive Elements,No Rating Summary,PSP,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,7


In [ ]:
df[df['platform']=='Plug-and-Play']

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year,generation
647,Bejeweled Deluxe TV Game,E,No Descriptors,No Interactive Elements,No Rating Summary,Plug-and-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,6
981,Radica Skannerz TV,E,Fantasy Violence,No Interactive Elements,No Rating Summary,Plug-and-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,6
1023,Dora's World Adventure,E,No Descriptors,No Interactive Elements,No Rating Summary,Plug-and-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,6
1077,"77300 TV Board Games 3-in-1 (Hungry Hippos, Si...",E,No Descriptors,No Interactive Elements,No Rating Summary,Plug-and-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,6
1402,Scooby Doo TV Games,E,No Descriptors,No Interactive Elements,No Rating Summary,Plug-and-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44848,Power Rangers S.P.D. - Escape of the Five Fugi...,E10+,Cartoon Violence,No Interactive Elements,No Rating Summary,Plug-and-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,6
44861,SpongeBob SquarePants - Sponge Pop/Snowball Sh...,E,No Descriptors,No Interactive Elements,No Rating Summary,Plug-and-Pl

In [ ]:
df[df['game_title']=="Borderlands"]

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year,generation
4831,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,PlayStation 3,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009.0,Take-Two Interactive,1.20,0.62,0.00,0.29,2.11,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,7
17731,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,Xbox 360,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009.0,Take-Two Interactive,2.39,0.73,0.03,0.29,3.44,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,7
23026,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,Xbox One,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009.0,NaN,NaN,NaN,NaN,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,8
25444,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,Windows PC,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009.0,Take-Two Interactive,0.01,0.14,0.00,0.04,0.18,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,9
39376,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,Nintendo Switch,NaN,NaN,NaN,NaN,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,9


In [ ]:
df[df['game_title']=="Borderlands"]

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,release_year,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,Number of Descriptors,Crude Humor,Mild Cartoon Violence,Cartoon Violence,Mild Language,Mild Violence,Violence,Blood and Gore,Language,Nudity,Sexual Themes,Mild Fantasy Violence,Alcohol Reference,Mild Suggestive Themes,Simulated Gambling,Fantasy Violence,Mild Blood,Use of Alcohol,Use of Tobacco,Drug Reference,Strong Language,Suggestive Themes,Blood,Partial Nudity,Tobacco Reference,Comic Mischief,Animated Blood,Intense Violence,Alcohol and Tobacco Reference,Violent References,Strong Sexual Content,Use of Alcohol and Tobacco,Use of Drugs,Mild Lyrics,Lyrics,Use of Drugs and Alcohol,Animated Violence,Mild Sexual Themes,Sexual Content,Drug and Alcohol Reference,Strong Lyrics,Mature Humor,Edutainment,Gambling,Animated Blood and Gore,Informational,Mature Sexual Themes,Mild Animated Violence,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood,release_date_year
6159,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,PlayStation 3,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009,Take-Two Interactive,1.20,0.62,0.00,0.29,2.11,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009
6160,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,PlayStation 3,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009,Take-Two Interactive,1.20,0.62,0.00,0.29,2.11,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009
7316,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,PlayStation 3,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009,Take-Two Interactive,1.20,0.62,0.00,0.29,2.11,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009
7317,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,PlayStation 3,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009,Take-Two Interactive,1.20,0.62,0.00,0.29,2.11,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009
22750,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions Not Rated by the ESRB.,This is a sci-fi first-person shooter in which...,Xbox 360,2009-10-20,2020-08-03T02:17:35,"2K, Gearbox Software","Action, Shooter, RPG",2009,Take-Two Interactive,2.39,0.73,0.03,0.29,3.44,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009
22751,Borderlands,M,"Blood and Gore,Intense Violence,Mature Humor,S...",Online Interactions

In [ ]:
for i in range(len(df)):
  df.loc[i,'release_year'] = int(df.loc[i,'release_year'])
  if pd.notna(df.at[i, 'global_sales']):
    df.loc[i,'na_sales'] = int(df.loc[i,'na_sales'])
    df.loc[i,'eu_sales'] = int(df.loc[i,'eu_sales'] )
    df.loc[i,'jp_sales'] = int(df.loc[i,'jp_sales'])
    df.loc[i,'other_sales'] = int(df.loc[i,'other_sales'])
    df.loc[i,'global_sales'] = int(df.loc[i,'global_sales'])


df['release_year'].value_counts()

,count
release_year,
2008,5441
2012,4671
2022,4173
2010,4155
2018,2833
2020,2335
2014,1965
2015,1953
2004,1899


In [ ]:
len(df[df['esrb_rating']=='AO'])

27

In [ ]:
# drop some columns?
# df = df.drop(['release_date_year', 'generation'], axis=1)
df

,game_title,esrb_rating,content_descriptors,interactive_elements,content_summary,platform,release_date,updated,developers,genres,...,Gaming,Realistic Violence,Realistic Blood,Suitable for Mature Users,Gambling Themes,Realistic Blood and Gore,Sexual Violence,Mild Realistic Violence,Suitable for All Users,Mild Animated Blood
0,"Ed, Edd n Eddy: Scam of the Century",E,"Crude Humor,Mild Cartoon Violence",No Interactive Elements,No Rating Summary,Nintendo DS,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Napoleon Dynamite,E10+,"Cartoon Violence,Mild Language",No Interactive Elements,No Rating Summary,Nintendo DS,2007-10-31,2019-01-09T12:41:06,7 Studios,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Rail Simulator,E,No Descriptors,No Interactive Elements,No Rating Summary,Windows PC,2007-10-12,2019-01-09T12:41:06,Kuju Entertainment,Simulation,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NES Open Tournament Golf,E,No Descriptors,No Interactive Elements,No Rating Summary,Wii,1987-06-14,2019-07-12T12:10:28,Nintendo,Sports,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ultimate Duck Hunting,T,Mild Violence,No Interactive Elements,No Rating Summary,Wii,2009-05-26,2019-01-09T12:41:06,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45886,Turok 3: Shadow of Oblivion,M,"Blood and Gore,Violence",No Interactive Elements,This is a retro-style first-person shooter in ...,PlayStation 4,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45887,Cabela's® Big Game Hunter: Pro Hunts,T,"Blood,Violence",No Interactive Elements,This is a hunting simulation game in which pla...,Wii U,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45888,JoJo's Bizarre Adventure: All-Star Battle,T,"Animated Blood,Language,Mild Suggestive Themes...",Online Interactions Not Rated by the ESRB.,This is a fighting game in which players battl...,PlayStation 3,2013-08-29,2019-09-17T11:05:54,BANDAI NAMCO Entertainment America,Fighting,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45889,Titanfall,M,"Blood and Gore,Strong Language,Violence",Online Interactions Not Rated by the ESRB.,This is a team-based first-person shooter in w...,Xbox One,2014-03-11,2020-11-30T09:29:04,"Electronic Arts, Respawn Entertainment, Bluepo...","Action, Shooter",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv('/content/drive/MyDrive/SQLproject/CombinedESRBdata.csv', index=False)